# Group API Project: Crime Rates and Unemployment Rates

## November 10, 2021 -- Jamie Mortensen, Heather Leighton-Dick


What if any relationship is there between national property crimes rates versus unemployment rates in 2010? in 2020?

Is there a noticeable difference between the years? Is there a potential correlation between crime and unemployment rates? Is there a stronger correlation in one subgroup of property crimes than the others?

### Definitions:

property crime: a category of crime, usually involving private property, that includes burglary, larceny, theft, motor vehicle theft, arson, shoplifting, and vandalism

burglary: entry into a building with the intent to commit a crime, most often theft, robbery, or murder

robbery: taking property unlawfully from a person or place by force or threat of force

larceny: the theft of physical items that are personal property, without threat of force (theft includes all variations on stealing property from another person or entity)

### Lessons Learned:

1) be careful of data that isn't labeled clearly, e.g. "value"

2) information literacy -- had to figure out what some numbers represented ("normalized x100,000")

3) new modules like "prettytable"

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from pprint import pprint
import citipy
import json
from citipy import citipy
import seaborn as sns
import prettytable



## Performing the API Calls and Collecting the Data